<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey_Model_Mark_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [26]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
#Import Even-Strength On-Ice Totals Data
skater_EV_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/EV%20On-Ice%20Totals.csv')
#print(skater_EV_totals_raw)
skater_EV_totals_raw.head()

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,57.90,57.33,56.89,...,54.34,3.16,2.22,0.06,10.82,14.68,17.36,0.94,7.59,89.76
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,54.75,53.16,53.30,53.14,...,1070.09,52.60,52.78,12.38,84.53,114.87,143.58,-0.18,10.03,90.59
2,Aaron Ekblad,20-21,FLA,D,35,630.15,57.67,54.51,53.65,53.97,...,481.39,26.51,21.58,6.70,54.32,59.26,83.07,4.93,7.68,93.24
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,62.97,58.90,59.83,59.35,...,857.55,61.71,41.78,29.17,214.91,307.62,394.70,19.93,9.96,91.61
4,Aaron Ness,19-20,ARI,D,24,292.63,49.44,51.47,49.69,49.55,...,276.12,10.93,10.53,-0.29,8.70,-2.64,-4.93,0.40,8.39,90.90


In [28]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_totals = skater_EV_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
skater_EV_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,58.74,3.01,2.95,3.16,2.22
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,54.75,49.91,71.36,58.98,52.60,52.78
2,Aaron Ekblad,20-21,FLA,D,35,630.15,57.67,55.13,25.20,18.50,26.51,21.58
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,62.97,59.63,70.82,41.65,61.71,41.78
4,Aaron Ness,19-20,ARI,D,24,292.63,49.44,50.93,12.82,13.11,10.93,10.53
...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,62.20,50.40,57.18,34.75,42.25,41.58
2476,Zdeno Chara,20-21,WSH,D,55,817.42,51.49,51.85,35.05,33.02,28.61,26.57
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,53.26,45.99,48.37,42.45,46.96,55.16
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,45.67,49.60,22.91,27.25,25.45,25.86


In [29]:
#Add columns for G+/-,xG+/-
skater_EV_totals['G+/-'] = skater_EV_totals['GF'] - skater_EV_totals['GA']
skater_EV_totals['xG+/-'] = skater_EV_totals['xGF'] - skater_EV_totals['xGA']
skater_EV_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,58.74,3.01,2.95,3.16,2.22,0.06,0.94
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,54.75,49.91,71.36,58.98,52.60,52.78,12.38,-0.18
2,Aaron Ekblad,20-21,FLA,D,35,630.15,57.67,55.13,25.20,18.50,26.51,21.58,6.70,4.93
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,62.97,59.63,70.82,41.65,61.71,41.78,29.17,19.93
4,Aaron Ness,19-20,ARI,D,24,292.63,49.44,50.93,12.82,13.11,10.93,10.53,-0.29,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,62.20,50.40,57.18,34.75,42.25,41.58,22.43,0.67
2476,Zdeno Chara,20-21,WSH,D,55,817.42,51.49,51.85,35.05,33.02,28.61,26.57,2.03,2.04
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,53.26,45.99,48.37,42.45,46.96,55.16,5.92,-8.20
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,45.67,49.60,22.91,27.25,25.45,25.86,-4.34,-0.41


In [30]:
skater_EV_totals = skater_EV_totals.drop(columns = ['GF%','xGF%','GF','GA','xGF','xGA'])
skater_EV_totals

,Player,Season,Team,Position,GP,TOI,G+/-,xG+/-
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40
...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41


In [31]:
skater_EV_totals = skater_EV_totals.rename(columns = {'Position':'Position_EV','GP':'GP_EV','TOI':'TOI_EV','G+/-':'G_EV','xG+/-':'xG_EV'})
#skater_EV_totals

In [32]:
skater_EV_totals.loc[skater_EV_totals.Player=="Andrew Cogliano"]

,Player,Season,Team,Position_EV,GP_EV,TOI_EV,G_EV,xG_EV
144,Andrew Cogliano,19-20,DAL,C,68,727.47,0.12,1.10
145,Andrew Cogliano,20-21,DAL,C,54,658.68,-9.46,2.91
146,Andrew Cogliano,21-22,COL/S.J,C,74,787.16,-23.06,-7.46


In [33]:
#Import PP On-Ice Rates Data
skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/PP%20On-Ice%20Totals.csv')
#print(skater_PP_totals_raw)
#skater_PP_totals_raw.head()

In [34]:
#Reduce PP On-Ice Totals Data
skater_PP_totals = skater_PP_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,93.16,10.31,0.87,9.81,0.72
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,92.26,21.66,1.93,18.23,1.53
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,91.76,35.45,2.87,29.39,2.64
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,90.73,7.40,0.99,8.91,0.91
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,77.83,13.47,2.95,11.76,3.35
...,...,...,...,...,...,...,...,...,...,...,...,...
1278,Zach Werenski,19-20,CBJ,D,63,171.17,80.98,87.14,17.46,4.10,17.75,2.62
1279,Zach Werenski,20-21,CBJ,D,33,73.60,100.00,85.02,4.99,0.00,5.56,0.98
1280,Zach Werenski,21-22,CBJ,D,64,215.48,87.27,81.96,25.51,3.72,24.45,5.38
1281,Zack Kassian,19-20,EDM,R,58,23.18,54.38,96.83,1.18,0.99,1.22,0.04


In [35]:
#Add columns for G+/-,xG+/-,G/s,xG/s
skater_PP_totals['G+/-'] = skater_PP_totals['GF'] - skater_PP_totals['GA']
skater_PP_totals['xG+/-'] = skater_PP_totals['xGF'] - skater_PP_totals['xGA']
#skater_PP_totals

In [36]:
#Add column for mean G+/- and xG+/-
G_mean = skater_PP_totals['G+/-'].mean()
xG_mean = skater_PP_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [37]:
#Add column for stats above average for G+/- and xG+/-
skater_PP_totals['GAA'] = skater_PP_totals['G+/-'] - G_mean
skater_PP_totals['xGAA'] = skater_PP_totals['xG+/-'] - xG_mean
#skater_PP_totals

In [38]:
#Drop columns
skater_PP_totals = skater_PP_totals.drop(columns = ['GF%','xGF%','GF','GA','xGF','xGA','G+/-','xG+/-','xGAA'])
#skater_PP_totals

In [39]:
skater_PP_totals = skater_PP_totals.rename(columns = {'Position':'Position_PP','GP':'GP_PP','TOI':'TOI_PP','GAA':'GAA_PP'})
#skater_PP_totals

In [40]:
#skater_PP_totals.loc[skater_PP_totals.Player=='Andrew Cogliano']

In [41]:
#Import SH On-Ice Totals Data
skater_SH_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/SH%20On-Ice%20Totals.csv')
#print(skater_SH_totals_raw)
#skater_SH_totals_raw.head()

In [42]:
#Reduce SH On-Ice Totals Data
skater_SH_totals = skater_SH_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
#skater_SH_totals

In [43]:
#Add columns for G+/-,xG+/-
skater_SH_totals['G+/-'] = skater_SH_totals['GF'] - skater_SH_totals['GA']
skater_SH_totals['xG+/-'] = skater_SH_totals['xGF'] - skater_SH_totals['xGA']
#skater_SH_totals

In [44]:
#Add column for mean G+/- and xG+/-
G_mean = skater_SH_totals['G+/-'].mean()
xG_mean = skater_SH_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [45]:
#Add column for stats above average for G+/- and xG+/-
skater_SH_totals['GAA'] = skater_SH_totals['G+/-'] - G_mean
skater_SH_totals['xGAA'] = skater_SH_totals['xG+/-'] - xG_mean
#skater_SH_totals

In [46]:
skater_SH_totals = skater_SH_totals.drop(columns = ['xGF%','xGF','xGA','xG+/-','xGAA','GF%','GF','GA','G+/-'])
#skater_SH_totals

In [47]:
skater_SH_totals = skater_SH_totals.rename(columns = {'Position':'Position_SH','GP':'GP_SH','TOI':'TOI_SH','GAA':'GAA_SH'})
#skater_SH_totals

In [48]:
skater_SH_totals.loc[skater_SH_totals.Player=='Andrew Cogliano']

,Player,Season,Team,Position_SH,GP_SH,TOI_SH,GAA_SH
68,Andrew Cogliano,19-20,DAL,C,67,98.72,-2.057407
69,Andrew Cogliano,20-21,DAL,C,52,77.60,2.762593
70,Andrew Cogliano,21-22,COL/S.J,C,73,130.12,-0.457407


In [49]:
#Import Player Box Stats
skater_box_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Stats%20Box.csv')
#print(skater_box_totals_raw)
#skater_box_totals_raw.head()

In [50]:
#Reduce Data Stats
skater_box_totals = skater_box_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G','A1','A2','Points']]
#skater_box_totals.head()

In [51]:
skater_box_totals.loc[skater_box_totals.Player=='Andrew Cogliano']

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points
144,Andrew Cogliano,19-20,DAL,C,68,848.10,3,9,2,14
145,Andrew Cogliano,20-21,DAL,C,54,752.02,5,4,2,11
146,Andrew Cogliano,21-22,COL/S.J,C,74,937.57,4,7,5,16


In [52]:
#Look at positions to get correct F/D average points
#print(skater_box_totals['Position'].unique())
#print(skater_box_totals.count())

forwards_table = skater_box_totals.loc[(skater_box_totals.Position == "L") | (skater_box_totals.Position == "C") | (skater_box_totals.Position == "C/L") | (skater_box_totals.Position == "R") | (skater_box_totals.Position == "L/R") | (skater_box_totals.Position == "C/R")]
#forwards_table.head(10)
#print(forwards_table.count())

defensemen_table = skater_box_totals.loc[(skater_box_totals.Position == "D") | (skater_box_totals.Position == "D/L")]
#defensemen_table.head(10)
#print(defensemen_table.count())

In [53]:
forwards_table.loc[forwards_table.Player=='Andrew Cogliano']

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points
144,Andrew Cogliano,19-20,DAL,C,68,848.10,3,9,2,14
145,Andrew Cogliano,20-21,DAL,C,54,752.02,5,4,2,11
146,Andrew Cogliano,21-22,COL/S.J,C,74,937.57,4,7,5,16


In [54]:
#Get Mean of Points
Points_mean = skater_box_totals['Points'].mean()
#print(Points_mean)

Forwards_mean = forwards_table['Points'].mean()
#print(Forwards_mean)

Defensemen_mean = defensemen_table['Points'].mean()
#print(Defensemen_mean)

In [55]:
#Get PAA
#skater_box_totals['PAA'] = skater_box_totals['Points'] - Points_mean
#skater_box_totals

#Get PAA for Fwds and Dmen and combine

forwards_table['PAA'] = forwards_table['Points'] - Forwards_mean
forwards_table.head(10)

defensemen_table['PAA'] = defensemen_table['Points'] - Defensemen_mean
defensemen_table.head(10)


#Add dataframes back together and sort again

concat_frames = [forwards_table, defensemen_table]
skater_box_totals = pd.concat(concat_frames)
skater_box_totals = skater_box_totals.sort_index()
#skater_box_totals.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [56]:
#Combine skater EV and PP stats
combined_player_df = pd.merge(skater_EV_totals, skater_PP_totals, how="outer", on=['Player','Season'])
combined_player_df.head()


,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G_EV,xG_EV,Team_y,Position_PP,GP_PP,TOI_PP,GAA_PP
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,NaN,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,FLA,D,67.0,95.90,-3.070109
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,FLA,D,35.0,132.75,7.219891
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,FLA,D,61.0,214.78,20.069891
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,NaN,NaN,NaN


In [57]:
skater_EV_PP = combined_player_df.drop(columns = ['Team_y','Position_PP','GP_PP'])
skater_EV_PP

,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67,NaN,NaN
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04,NaN,NaN
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20,NaN,NaN
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41,NaN,NaN


In [58]:
skater_EV_PP.loc[skater_EV_PP.Player=='Andrew Cogliano']

,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP
144,Andrew Cogliano,19-20,DAL,C,68,727.47,0.12,1.10,NaN,NaN
145,Andrew Cogliano,20-21,DAL,C,54,658.68,-9.46,2.91,NaN,NaN
146,Andrew Cogliano,21-22,COL/S.J,C,74,787.16,-23.06,-7.46,NaN,NaN


In [59]:
#Merge EV,PP,SH tables together
full_player_totals = pd.merge(skater_EV_PP, skater_SH_totals, how="left", on=["Player", "Season"])
full_player_totals.head(10)

#full_player_totals = pd.concat([skater_EV_PP,skater_SH_totals], axis='columns')
#full_player_totals.head(20)

,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,Team,Position_SH,GP_SH,TOI_SH,GAA_SH
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,FLA,D,65.0,134.87,-13.197407
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,FLA,D,33.0,86.47,2.662593
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,FLA,D,61.0,166.83,-1.627407
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Adam Boqvist,19-20,CHI,D,41,563.95,3.21,-1.85,87.02,-6.100109,NaN,NaN,NaN,NaN,NaN
6,Adam Boqvist,20-21,CHI,D,35,459.97,-2.34,-3.11,118.65,-1.990109,NaN,NaN,NaN,NaN,NaN
7,Adam Boqvist,21-22,CBJ,D,52,763.68,-3.32,-7.46,98.63,-4.540109,NaN,NaN,NaN,NaN,NaN
8,Adam Brooks,19-20,TOR,C,7,51.60,1.95,0.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Adam Brooks,20-21,TOR,C,11,110.38,3.13,0.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
#Clean up data
full_player_totals = full_player_totals.drop(columns = ['Team','Position_SH','GP_SH'])
full_player_totals.head(10)

,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,NaN,NaN
5,Adam Boqvist,19-20,CHI,D,41,563.95,3.21,-1.85,87.02,-6.100109,NaN,NaN
6,Adam Boqvist,20-21,CHI,D,35,459.97,-2.34,-3.11,118.65,-1.990109,NaN,NaN
7,Adam Boqvist,21-22,CBJ,D,52,763.68,-3.32,-7.46,98.63,-4.540109,NaN,NaN
8,Adam Brooks,19-20,TOR,C,7,51.60,1.95,0.35,NaN,NaN,NaN,NaN
9,Adam Brooks,20-21,TOR,C,11,110.38,3.13,0.49,NaN,NaN,NaN,NaN


In [61]:
full_player_totals.loc[full_player_totals.Player=='Andrew Cogliano']

,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH
144,Andrew Cogliano,19-20,DAL,C,68,727.47,0.12,1.10,NaN,NaN,98.72,-2.057407
145,Andrew Cogliano,20-21,DAL,C,54,658.68,-9.46,2.91,NaN,NaN,77.60,2.762593
146,Andrew Cogliano,21-22,COL/S.J,C,74,787.16,-23.06,-7.46,NaN,NaN,130.12,-0.457407


In [62]:
#Merge EV,PP,SH and Points tables together
full_player_totals = pd.merge(full_player_totals, skater_box_totals, how="left", on=["Player", "Season"])
full_player_totals.head()

,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,...,GAA_SH,Team,Position,GP,TOI,G,A1,A2,Points,PAA
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,...,NaN,N.J,L,9,70.70,1,0,1,2,-22.347988
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,...,-13.197407,FLA,D,67,1537.92,5,23,13,41,26.004535
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,...,2.662593,FLA,D,35,878.18,11,5,6,22,7.004535
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,...,-1.627407,FLA,D,61,1519.70,15,13,29,57,42.004535
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,...,NaN,ARI,D,24,300.83,0,1,0,1,-13.995465


In [63]:
#Drop Repeated Columns
full_player_totals = full_player_totals.drop(columns = ['Team','Position','GP','G','A1','A2'])
full_player_totals.head(10)


,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI,Points,PAA
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,NaN,NaN,70.70,2,-22.347988
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407,1537.92,41,26.004535
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593,878.18,22,7.004535
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407,1519.70,57,42.004535
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,NaN,NaN,300.83,1,-13.995465
5,Adam Boqvist,19-20,CHI,D,41,563.95,3.21,-1.85,87.02,-6.100109,NaN,NaN,665.12,13,-1.995465
6,Adam Boqvist,20-21,CHI,D,35,459.97,-2.34,-3.11,118.65,-1.990109,NaN,NaN,594.65,16,1.004535
7,Adam Boqvist,21-22,CBJ,D,52,763.68,-3.32,-7.46,98.63,-4.540109,NaN,NaN,886.05,22,7.004535
8,Adam Brooks,19-20,TOR,C,7,51.60,1.95,0.35,NaN,NaN,NaN,NaN,54.97,3,-21.347988
9,Adam Brooks,20-21,TOR,C,11,110.38,3.13,0.49,NaN,NaN,NaN,NaN,117.45,5,-19.347988


In [64]:
#Rename Columns
full_player_totals = full_player_totals.rename(columns = {'Team_x':'Team','Position_EV':'Position','GP_EV':'GP','TOI':'TOI_tot'})
full_player_totals

,Player,Season,Team,Position,GP,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI_tot,Points,PAA
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,NaN,NaN,70.70,2,-22.347988
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407,1537.92,41,26.004535
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593,878.18,22,7.004535
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407,1519.70,57,42.004535
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,NaN,NaN,300.83,1,-13.995465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67,NaN,NaN,214.07,-11.567407,1428.48,14,-0.995465
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04,NaN,NaN,145.72,-6.487407,1006.27,10,-4.995465
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20,NaN,NaN,123.65,4.072593,1348.45,14,-0.995465
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41,NaN,NaN,147.87,-13.037407,951.77,19,-5.347988


In [65]:
#Replace NaN with 0
full_player_totals = full_player_totals.replace(np.NaN, 0)
full_player_totals

,Player,Season,Team,Position,GP,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI_tot,Points,PAA
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,0.00,0.000000,0.00,0.000000,70.70,2,-22.347988
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407,1537.92,41,26.004535
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593,878.18,22,7.004535
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407,1519.70,57,42.004535
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,0.00,0.000000,0.00,0.000000,300.83,1,-13.995465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67,0.00,0.000000,214.07,-11.567407,1428.48,14,-0.995465
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04,0.00,0.000000,145.72,-6.487407,1006.27,10,-4.995465
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20,0.00,0.000000,123.65,4.072593,1348.45,14,-0.995465
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41,0.00,0.000000,147.87,-13.037407,951.77,19,-5.347988


In [66]:
full_player_totals.loc[full_player_totals.Player=='Andrew Cogliano']

,Player,Season,Team,Position,GP,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI_tot,Points,PAA
144,Andrew Cogliano,19-20,DAL,C,68,727.47,0.12,1.10,0.0,0.0,98.72,-2.057407,848.10,14,-10.347988
145,Andrew Cogliano,20-21,DAL,C,54,658.68,-9.46,2.91,0.0,0.0,77.60,2.762593,752.02,11,-13.347988
146,Andrew Cogliano,21-22,COL/S.J,C,74,787.16,-23.06,-7.46,0.0,0.0,130.12,-0.457407,937.57,16,-8.347988


In [67]:
#Import Goalie Data
goalie_stats_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Goalie%20Stats.csv')
#print(goalie_stats_raw)
goalie_stats_raw.head()

,Player,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx
0,Aaron Dell,19-20,S.J,G,33,1834.23,91.06,994.37,1352.10,92.16,90.84,93.27,93.18,0.08,-1.20,1.10
1,Aaron Dell,20-21,N.J,G,7,319.17,21.45,153.29,210.23,11.91,86.01,89.80,94.33,-4.54,-7.35,-9.54
2,Aaron Dell,21-22,BUF,G,12,565.38,37.95,356.65,467.99,30.60,89.36,91.89,93.46,-1.57,-4.85,-7.35
3,Adam Huska,21-22,NYR,G,1,59.70,7.53,42.56,55.97,3.50,82.31,86.55,93.75,-7.20,-3.58,-4.03
4,Adam Werner,19-20,COL,G,2,87.83,4.85,54.22,65.83,3.76,91.05,92.63,94.29,-1.66,0.05,-1.09


In [68]:
#Reduce Goalie Stats
goalie_stats = goalie_stats_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GA','Sv%','GSAx']]
goalie_stats.head()

,Player,Season,Team,Position,GP,TOI,GA,Sv%,GSAx
0,Aaron Dell,19-20,S.J,G,33,1834.23,91.06,90.84,1.10
1,Aaron Dell,20-21,N.J,G,7,319.17,21.45,86.01,-9.54
2,Aaron Dell,21-22,BUF,G,12,565.38,37.95,89.36,-7.35
3,Adam Huska,21-22,NYR,G,1,59.70,7.53,82.31,-4.03
4,Adam Werner,19-20,COL,G,2,87.83,4.85,91.05,-1.09


In [70]:
#Add Active / Inactive Column to both data tables
full_player_totals['A/I'] = 'Active'
full_player_totals

,Player,Season,Team,Position,GP,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI_tot,Points,PAA,A/I
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,0.00,0.000000,0.00,0.000000,70.70,2,-22.347988,Active
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407,1537.92,41,26.004535,Active
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593,878.18,22,7.004535,Active
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407,1519.70,57,42.004535,Active
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,0.00,0.000000,0.00,0.000000,300.83,1,-13.995465,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67,0.00,0.000000,214.07,-11.567407,1428.48,14,-0.995465,Active
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04,0.00,0.000000,145.72,-6.487407,1006.27,10,-4.995465,Active
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20,0.00,0.000000,123.65,4.072593,1348.45,14,-0.995465,Active
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41,0.00,0.000000,147.87,-13.037407,951.77,19,-5.347988,Active


In [71]:
#Make a Data Table for each team
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
#Avalanche_skaters.head(20)

,Player,Season,Team,Position,GP,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI_tot,Points,PAA,A/I
84,Alex Newhook,20-21,COL,C,6,74.70,5.32,2.29,0.00,0.000000,0.00,0.000000,82.07,3,-21.347988,Active
85,Alex Newhook,21-22,COL,C,71,852.80,12.63,0.07,97.58,-3.930109,0.00,0.000000,963.07,33,8.652012,Active
125,Andre Burakovsky,19-20,COL,L,58,724.50,15.47,-0.48,136.92,-2.060109,0.00,0.000000,881.17,45,20.652012,Active
126,Andre Burakovsky,20-21,COL,L,53,675.22,6.57,10.53,113.35,5.669891,0.00,0.000000,799.88,44,19.652012,Active
127,Andre Burakovsky,21-22,COL,L,80,1125.07,12.74,5.04,157.03,5.559891,0.00,0.000000,1301.25,61,36.652012,Active
261,Bowen Byram,20-21,COL,D,19,309.83,3.11,2.30,0.00,0.000000,0.00,0.000000,332.72,2,-12.995465,Active
262,Bowen Byram,21-22,COL,D,30,522.07,5.26,0.96,29.42,-11.630109,0.00,0.000000,564.23,17,2.004535,Active
294,Brandon Saad,20-21,COL,L,44,531.95,2.38,6.71,73.55,-5.230109,0.00,0.000000,619.50,24,-0.347988,Active
382,Cale Makar,19-20,COL,D,57,954.60,22.63,5.83,210.18,12.119891,0.00,0.000000,1197.60,50,35.004535,Active
383,Cale Makar,20-21,COL,D,44,823.45,23.13,14.57,184.18,17.439891,39.60,8.732593,1069.57,44,29.004535,Active


In [ ]:
full_player_totals.head(30)

In [ ]:
#Inspect Data
full_player_totals.head(20)

In [ ]:
#skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/PP%20On-Ice%20Rates.csv')
#print(skater_PP_rates_raw)
#skater_PP_rates_raw.head()

**Stuff to Do Next - (have no more time, so quick thoughts that I have not checked, like the next one would be so easy to check if I had another 5 mins)**

In [ ]:
#Are these right? duplicate naming issues for "TOI_SH" and no PP?
full_player_totals.columns

Need to create a dataframe for each team. Use the loc function on player table first to get all the players for one team. Then do the same thing with goalies. We will probably need to keep them seperate tables since they do not have similar variables

Not necessary at this stage but eventually we are going to want to consolidate these code blocks. Ex, the code to create all of the team rosters should be in one block with no output. For now you can do 64 blocks (32 for skaters, 32 for goalies) and show output so we can see if there are any errors, but once we get past that stage we will comment out the output and put it all in one block. If we ever need to go back in check you just take out the '#' and run the code to see the output.

Need to add a column for 'status'. I am thinking the two states we use are 'active' and 'inactive' for the starting lineups. Will be 18 skaters and 1 goalie when we update daily but doesnt matter now. Should put this in before all of the loc functions so it shows up for the team dataframes. Note: will have to be added to both the full_player_totals dataframe and the goalie_stats dataframe.

Add folder in github for the csv's, update github links in code to pull data